In [26]:
from wikidataintegrator import wdi_core, wdi_login
from wikidataintegrator.ref_handlers import update_retrieved_if_new_multiple_refs
import pandas as pd
import numpy as np
import requests
from datetime import datetime
from time import sleep
from random import random

In [28]:
kantonZH = pd.read_csv('kantonZH.csv', parse_dates=['date'])    
kantonZH.head()
#[(city['Name']=='Adlikon')]
cityZH = pd.read_csv('cityZH.csv', parse_dates=['date'])    
cityZH.head()
df = cityZH

print("Logging in...")
#if "WDUSER" in os.environ and "WDPASS" in os.environ:
#    WDUSER = os.environ['WDUSER']
#    WDPASS = os.environ['WDPASS']
#else:
#    raise ValueError("WDUSER and WDPASS must be specified in local.py or as environment variables")
login = wdi_login.WDLogin("Tlzh", "wikiwiki")

Logging in...
https://www.wikidata.org/w/api.php
Successfully logged in as Tlzh


In [29]:
# Quelle setzen
# Quartier 
#  Q4115189|P1082|805|P585|+2017-12-31T00:00:00Z/9|P459|Q12483|S854|"https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_seit1970_od3240"
from datetime import datetime
import copy
def create_reference():
    timeStringNow = datetime.now().strftime("+%Y-%m-%dT00:00:00Z")
    # Timestamp P585
    refRetrieved = wdi_core.WDTime(timeStringNow, prop_nr="P813", is_reference=True)
    # URL Source P854
    refURL = wdi_core.WDUrl(value="https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_seit1970_od3240", 
                            prop_nr="P854", is_reference=True)

    return [refRetrieved, refURL]

def create_qualifier(insertDate):
    # Point in time P585
    insertDate = insertDate.strftime("+%Y-%m-%dT00:00:00Z")
    refRetrieved = wdi_core.WDTime(insertDate, prop_nr="P585", is_qualifier=True)
    # Statistics P459
    bestimmungsmethode = wdi_core.WDItemID(value='Q12483', prop_nr='P459', is_qualifier=True)

    return [refRetrieved, bestimmungsmethode]


In [30]:
df = df[df['bool']==False]
for i, row in df.iterrows():
    print("Insert data value: {0} - {1} - {2} : {3}".format(row['wikidata_id'],row['name'], row['date'], row['population']))
    reference = create_reference()
    insertDate = datetime(2017,12,31).strftime("+%Y-%m-%dT00:00:00Z")
    qualfier = create_qualifier(row['date'])
    statement = [wdi_core.WDQuantity(value=row['population'], prop_nr="P1082", references=[copy.deepcopy(reference)], 
                                     rank='normal', qualifiers=copy.deepcopy(qualfier), check_qualifier_equality=False)]
    wd_item = wdi_core.WDItemEngine(data=statement,wd_item_id=row['wikidata_id'], new_item=False, fast_run=False, append_value=['P1082'])
    wd_item.write(login)
    sleep(8 + random())

Insert data value: Q455496 - Höngg - 2010-12-31 00:00:00 : 21179
Insert data value: Q455496 - Höngg - 2009-12-31 00:00:00 : 21294
Insert data value: Q455496 - Höngg - 2008-12-31 00:00:00 : 21186
Insert data value: Q455496 - Höngg - 2007-12-31 00:00:00 : 21075
Insert data value: Q455496 - Höngg - 2006-12-31 00:00:00 : 21017
Insert data value: Q455496 - Höngg - 2005-12-31 00:00:00 : 20773
Insert data value: Q455496 - Höngg - 2004-12-31 00:00:00 : 20856
Insert data value: Q455496 - Höngg - 2003-12-31 00:00:00 : 20810
Insert data value: Q455496 - Höngg - 2002-12-31 00:00:00 : 20570
Insert data value: Q455496 - Höngg - 2001-12-31 00:00:00 : 20588
Insert data value: Q455496 - Höngg - 2000-12-31 00:00:00 : 20423
Insert data value: Q455496 - Höngg - 1999-12-31 00:00:00 : 20559
Insert data value: Q455496 - Höngg - 1998-12-31 00:00:00 : 20054
Insert data value: Q455496 - Höngg - 1997-12-31 00:00:00 : 19793
Insert data value: Q455496 - Höngg - 1996-12-31 00:00:00 : 19604
Insert data value: Q45549